## Análise dos dados dos covenants coletados

In [34]:
# bibliotecas usadas
import os
import re
import fitz  # PyMuPDF
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [35]:
df_covenants.columns

Index(['EMPRESA', 'POSSUI COVENANT', 'DIVULGOU', 'ANO',
       'Debenture ou emprestimo e financiamento', 'TIPO', 'Limite', 'Violou?',
       'Unnamed: 8', 'Texto Extraído', 'Texto Processado'],
      dtype='object')

In [36]:
# **1. Carregar e pré-processar o CSV**
df_covenants = pd.read_csv("base_covenants.csv", delimiter=";", encoding="utf-8")
df_covenants.columns = df_covenants.columns.str.strip()

# **2. Criar colunas vazias para processamento**
df_covenants["Texto Extraído"] = np.nan
df_covenants["Texto Processado"] = np.nan

# **3. Função para extrair texto do PDF**
def extrair_texto_pdf(caminho_pdf):
    """ Extrai texto de um PDF """
    texto = ""
    with fitz.open(caminho_pdf) as pdf:
        for pagina in pdf:
            texto += pagina.get_text("text") + "\n"
    return texto.strip()

# **4. Função de limpeza do texto**
def preprocessar_texto(texto):
    """ Limpa e tokeniza removendo pontuações e palavras irrelevantes """
    texto = texto.lower()
    texto = re.sub(r'[^\w\s]', ' ', texto)  # Remove caracteres especiais
    stopwords = {"a", "e", "o", "de", "do", "da", "em", "para", "por", "com", "um", "uma", "os", "as"}  # Stopwords básicas
    tokens = [palavra for palavra in texto.split() if palavra not in stopwords]
    return " ".join(tokens)

In [37]:
# **5. Processar os PDFs**
pasta_base = r"M:\Maria Eduarda\Downloads\Empresas Com Covenants"

for i, row in df_covenants.iterrows():
    empresa = row['EMPRESA']
    ano = row['ANO']
    
    # Verificar se o ano é NaN
    if pd.isna(ano):
        print(f"⚠️ Ano inválido para a empresa {empresa}, linha {i}")
        continue

    # Converter o ano para inteiro
    ano = int(ano)

    # Construir o caminho correto do PDF
    caminho_pdf = os.path.join(pasta_base, empresa, f"{ano}.pdf", f"{ano}.pdf")
    
    if os.path.exists(caminho_pdf):
        texto_extraido = extrair_texto_pdf(caminho_pdf)
        df_covenants.at[i, "Texto Extraído"] = texto_extraido
        df_covenants.at[i, "Texto Processado"] = preprocessar_texto(texto_extraido)
    else:
        print(f"⚠️ Arquivo não encontrado: {caminho_pdf}")

# **6. Vetorização TF-IDF**
vectorizer = TfidfVectorizer(max_features=1000, stop_words="english")
X = vectorizer.fit_transform(df_covenants["Texto Processado"].fillna(""))


⚠️ Arquivo não encontrado: M:\Maria Eduarda\Downloads\Empresas Com Covenants\Aeris Indústria e Comércio de Equipamentos para Geração de Energia S.A.\2019.pdf\2019.pdf
⚠️ Arquivo não encontrado: M:\Maria Eduarda\Downloads\Empresas Com Covenants\Aeris Indústria e Comércio de Equipamentos para Geração de Energia S.A.\2020.pdf\2020.pdf
⚠️ Arquivo não encontrado: M:\Maria Eduarda\Downloads\Empresas Com Covenants\Aeris Indústria e Comércio de Equipamentos para Geração de Energia S.A.\2021.pdf\2021.pdf
⚠️ Arquivo não encontrado: M:\Maria Eduarda\Downloads\Empresas Com Covenants\Aeris Indústria e Comércio de Equipamentos para Geração de Energia S.A.\2022.pdf\2022.pdf
⚠️ Arquivo não encontrado: M:\Maria Eduarda\Downloads\Empresas Com Covenants\Aeris Indústria e Comércio de Equipamentos para Geração de Energia S.A.\2023.pdf\2023.pdf


C:\Users\Maria Eduarda\AppData\Local\Temp\ipykernel_3356\3691495334.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'BALANÇOS PATRIMONIAIS
31 de dezembro de 2021 e 2020
(Valores expressos em milhares de reais – R$)
Reapresentado (i)
Reapresentado (i)
Controladora
Consolidado
Notas
2021
2020
2021
2020
ATIVO CIRCULANTE
Caixa e equivalentes de caixa
a
1
5
 
153.035 
 
492.657 
 
595.784 
 
504.914 
Investimentos de curto prazo
o
1
5
 
271.101 
 
1.035.044 
 
1.055.269 
 
1.522.889 
Contas a receber de clientes
s
1
6
 
503.120 
 
182.501 
 
364.560 
 
286.571 
Imposto de renda e contribuição social a 
recuperar
a
r
1
7
 
66.909 
 
27.907 
 
73.049 
 
30.277 
Outros tributos a recuperar
r
1
7
 
111 
 
425 
 
1.403 
 
4.553 
Instrumentos financeiros derivativos
s
1
34
 
— 
 
— 
 
3.547 
 
— 
Cauções e depósitos vinculados
s
1
9
 
33.189 
 
16.908 
 
60.293 
 
22.615 
Conta de ressarcimento
o
1
10
 
— 
 
— 

⚠️ Ano inválido para a empresa AES Brasil Energia, linha 6
⚠️ Ano inválido para a empresa AES Brasil Energia, linha 7
⚠️ Ano inválido para a empresa AES Brasil Energia, linha 8
⚠️ Ano inválido para a empresa AES Brasil Energia, linha 10
⚠️ Ano inválido para a empresa AES Brasil Energia, linha 11
⚠️ Ano inválido para a empresa AES Brasil Energia, linha 12
⚠️ Ano inválido para a empresa AES Brasil Energia, linha 13
⚠️ Ano inválido para a empresa AES Brasil Energia, linha 14
⚠️ Ano inválido para a empresa AES Brasil Energia, linha 15
⚠️ Ano inválido para a empresa AES Brasil Energia, linha 16
⚠️ Ano inválido para a empresa AES Brasil Energia, linha 17
⚠️ Ano inválido para a empresa AES Brasil Energia, linha 18
⚠️ Ano inválido para a empresa AES Brasil Energia, linha 19
⚠️ Ano inválido para a empresa AES Brasil Energia, linha 20
⚠️ Ano inválido para a empresa AES Brasil Energia, linha 21
⚠️ Ano inválido para a empresa AES Brasil Energia, linha 22
⚠️ Ano inválido para a empresa AES Brasil E

KeyboardInterrupt: 

In [ ]:
# **6. Vetorização TF-IDF**
vectorizer = TfidfVectorizer(max_features=1000, stop_words="english")
X = vectorizer.fit_transform(df_covenants["Texto Processado"].fillna(""))

# **7. Codificar labels**
encoder_violacao = LabelEncoder()
encoder_divulgacao = LabelEncoder()
encoder_tipo = LabelEncoder()
df_covenants["Violou?"] = df_covenants["Violou?"].fillna("Não")
df_covenants["DIVULGOU"] = df_covenants["DIVULGOU"].fillna("Não")
df_covenants["TIPO"] = df_covenants["TIPO"].fillna("Desconhecido")
y_violacao = encoder_violacao.fit_transform(df_covenants["Violou?"])
y_divulgacao = encoder_divulgacao.fit_transform(df_covenants["DIVULGOU"])
y_tipo = encoder_tipo.fit_transform(df_covenants["TIPO"])


[nltk_data] Downloading package stopwords to C:\Users\Maria
[nltk_data]     Eduarda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
# **8. Treinar modelos de classificação**
X_train, X_test, y_train_violacao, y_test_violacao = train_test_split(X, y_violacao, test_size=0.2, random_state=42)
X_train, X_test, y_train_divulgacao, y_test_divulgacao = train_test_split(X, y_divulgacao, test_size=0.2, random_state=42)
X_train, X_test, y_train_tipo, y_test_tipo = train_test_split(X, y_tipo, test_size=0.2, random_state=42)

modelo_violacao = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_violacao.fit(X_train, y_train_violacao)

modelo_divulgacao = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_divulgacao.fit(X_train, y_train_divulgacao)

modelo_tipo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_tipo.fit(X_train, y_train_tipo)

# **9. Prever colunas automaticamente**
df_covenants["Previsão Violação"] = encoder_violacao.inverse_transform(modelo_violacao.predict(X))
df_covenants["Previsão Divulgação"] = encoder_divulgacao.inverse_transform(modelo_divulgacao.predict(X))
df_covenants["Previsão Tipo"] = encoder_tipo.inverse_transform(modelo_tipo.predict(X))


In [ ]:
# **10. Extrair Limite Financeiro (Expressões Numéricas)**
def extrair_limite(texto):
    """ Procura por expressões numéricas que indicam limites financeiros """
    match = re.search(r"(\d+(\.\d+)?)\s*x", texto)  # Exemplo: "Dívida líquida/EBITDA ≤ 3,0x"
    if match:
        return match.group(1)
    return "Desconhecido"

df_covenants["Limite Estimado"] = df_covenants["Texto Extraído"].apply(lambda x: extrair_limite(x) if isinstance(x, str) else "Desconhecido")


In [ ]:
# **11. Salvar resultados**
df_covenants.to_csv("covenants_analisados.csv", index=False, encoding="utf-8")
print("✅ Análise concluída e salva como 'covenants_analisados.csv'")